In [45]:
from hello import hello3
from matplotlib import pyplot as plt
import numpy
from officelib.xllib import *
import datetime

In [28]:
def date(ts):
    return datetime.datetime.fromtimestamp(ts/1000).strftime("%m/%d/%y %I:%M:%S %p")

def td2xld(data):
    td = data['last']
    l = [("Time", "DOPV(%)")]
    l.extend((date(ts), pv) for ts, pv in td)
    return l

def paste(cells, xld):
    tl = cells.Range("A1")
    tr = tl.Offset(1, len(xld[0]))
    br = tr.Offset(len(xld), 1)
    cells.Range(tl, br).Value = xld
    

In [31]:
def main():
    ip = '71.189.82.196:85'
    h = hello3.HelloApp(ip)
    data = h.getTrendData('12hr', 'do')
    xld = td2xld(data)
    xl, wb, ws, cells = xlObjs()
    paste(cells, xld)
    wb.Name = "Foo.Bar"

In [43]:
xl = Excel()
for f in xl.RecentFiles:
    if f.Name.lower().startswith("80l do decay"):
        wb = f.Open()
        break
else:
    raise ValueError("uhoh")
ws = wb.Worksheets(1)
cells = ws.Cells

In [44]:
xc = cells.Range("BR1")
yc = cells.Range("BT1")
data = []
topleft = cells.Range("BV6")
with screen_lock(xl):
    for i in range(54400, 54600, 1):
        x = i/10000
        if not i % 100:
            print("\rtesting x=%.3f"%x, end="")
        xc.Value = x
        y = yc.Value
        data.append((x,y))
    br = topleft.Offset(len(data), 2)
    cells.Range(topleft, br).Value = data
    print("done")

testing x=5.450done


In [139]:
from officelib.const import xlconst as xlc
from pywintypes import com_error
xl = Excel()

wb = xl.ActiveWorkbook
if not wb:
    wb = xl.RecentFiles(1).Open()
ws1 = wb.Worksheets(1)
try:
    ws2 = wb.Worksheets(2)
except com_error:
    ws2 = wb.Worksheets.Add()
wb.Name

'Overnight Stability Test 170215.xlsb'

In [140]:
ws2.UsedRange.Clear()
max_data = 20000
with screen_lock(xl):
    c1 = ws1.Cells.Range("A1")
    c2 = ws2.Cells.Range("A1")
    while True:
        print("\rExtracting data from %s     "%c1.GetAddress(0,0), end="")
        v = c1.Value
        if v == "Start Time":
            break
        tl = c1
        br = c1.End(xlc.xlDown).Offset(1,2)
        data = ws1.Cells.Range(tl, br).Value
        if len(data) > max_data:
            skip = len(data) // max_data + 1
        data = data[::skip]

        ptl = c2
        pbr = c2.Offset(len(data), 2)
        ws2.Cells.Range(ptl, pbr).Value = data

        c1 = c1.Offset(1, 4)
        c2 = c2.Offset(1, 4)
print("\rDone                            ")

Done                            


In [141]:
def getref():
    dopv = ws2.Cells.Find(What="DOPV(%)", SearchOrder=xlc.xlByRows)
    ref = dopv.Offset(2, 1)
    factor = dopv.Offset(1,2)
    minus = dopv.Offset(1, 4)
    return ref, factor, minus
dopv = ws2.Cells.Find(What="DOPV(%)", SearchOrder=xlc.xlByRows)
dopv.Offset(1,2).EntireColumn.Insert()

True

In [142]:
def elapsed(cells, c, insert=True): 
    if insert:
        c.Offset(1,2).EntireColumn.Insert()
    #ref, factor = getref()
    form = "=(%s-%s)*%s - %s" %(c.GetAddress(0,0), ref.Address, 
                                factor.Address, minus.Address)
    ar = cells.Range(c.Offset(1,2), c.End(xlc.xlDown).Offset(1,2))
    ar.NumberFormat = "0.00"
    c.Offset(1,2).Value = form
    c.Offset(1,2).AutoFill(ar)
    
c1 = ws2.Cells.Range("A1")
with screen_lock(xl):
    while True:
        ref, factor, minus = getref()
        if c1.Value == "DOPV(%)":
            insert = False
        else:
            insert = True
        if c1.Value is None:
            break
        elapsed(ws2.Cells, c1.Offset(2,1), insert)
        c1 = c1.Offset(1,5)
factor.Value = 24
minus.Value = 0

In [150]:
c1, c2, c3 = wb.Worksheets("Sheet1").ChartObjects()

In [155]:
class myint(int):
    def __bool__(self):
        return True
for c in c1, c2, c3:
    FormatAxesScale(c.Chart, myint(0), 16)
    